<a href="https://colab.research.google.com/github/reolingovender/MIT805/blob/main/Image_Analysis_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
pip install datasets

In [29]:
# install msodbcsql18
%%sh
apt-get install -y unixodbc-dev
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql18

Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
OK
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 htt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   2897      0 --:--:-- --:--:-- --:--:--  2899
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    362      0 --:--:-- --:--:-- --:--:--   363
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [30]:
!pip install pyodbc

In [31]:
pip install pyspark


In [32]:
import requests

ip = requests.get('https://api64.ipify.org?format=json').json()['ip']
print(f'Your Colab IP address is: {ip}')

Your Colab IP address is: 34.139.152.215


In [33]:
import datasets
from datasets import load_dataset
import numpy as np
import pandas as pd

In [34]:
# Initialize Spark session
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder.appName("Optimized PySpark").getOrCreate()

In [35]:
datasets.logging.set_verbosity_error()
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)

In [36]:
df = pd.DataFrame(dataset['full'])

In [37]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [38]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType, IntegerType, MapType, LongType, BooleanType, FloatType
from pyspark.sql.functions import col, explode, expr, collect_list, when

# Define the schema, specifying the type for the `images` field and other fields.
schema_review = StructType([
    StructField("rating", FloatType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("images", ArrayType(MapType(StringType(), StringType())), True),  # Array of dictionaries
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("helpful_vote", IntegerType(), True),
    StructField("verified_purchase", BooleanType(), True)
])


In [39]:
# Convert Pandas DataFrames to PySpark DataFrames
pyspark_df_reviews = spark.createDataFrame(df, schema = schema_review)

In [40]:
# Filter reviews that had no images
pyspark_df_reviews = pyspark_df_reviews.withColumn('number_images', f.size('images')).withColumn("id", f.monotonically_increasing_id())

# Count how mnay images reviews had
pyspark_df_reviews = pyspark_df_reviews.filter("number_images != 0").filter("rating > 0 and rating is not null")

# Extract the 'large_image_url' from the 'images' column
pyspark_df_reviews = pyspark_df_reviews.withColumn('large_image_url', expr("images[0]['large_image_url']"))

# Explode the 'images' array to get each dictionary as a separate row
exploded_df = pyspark_df_reviews.select(explode(col("images")).alias("image"))

# Extract 'large_image_url' from each exploded dictionary
extracted_df = exploded_df.select(col("image")["large_image_url"].alias("large_image_url"))

# Collect all 'large_image_url' values back into a list per original row
pyspark_df_reviews = pyspark_df_reviews.withColumn("all_large_image_urls", expr("transform(images, x -> x['large_image_url'])"))

# Convert 'timestamp' to correct formatting
pyspark_df_reviews.withColumn("timestamp", f.date_format(f.from_unixtime(col("timestamp") / 1000), "yyyy-MM-dd")).show()

+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+-----------------+-------------+---+--------------------+--------------------+
|rating|               title|                text|              images|      asin|parent_asin|             user_id| timestamp|helpful_vote|verified_purchase|number_images| id|     large_image_url|all_large_image_urls|
+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+-----------------+-------------+---+--------------------+--------------------+
|   4.0|        Pretty Color|The polish was qu...|[{large_image_url...|B00R8DXL44| B00R8DXL44|AGMJ3EMDVL6OWBJF7...|2020-08-27|           0|             true|            1|  5|https://images-na...|[https://images-n...|
|   3.0|                 Meh|These were lightw...|[{large_image_url...|B088SZDGXG| B08BBQ29N5|AEYORY2AVPMCPDV57...|2021-10-15|  

In [41]:
# Show the final DataFrame
pyspark_df_reviews.show()

+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+-------------+---+--------------------+--------------------+
|rating|               title|                text|              images|      asin|parent_asin|             user_id|    timestamp|helpful_vote|verified_purchase|number_images| id|     large_image_url|all_large_image_urls|
+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+-------------+---+--------------------+--------------------+
|   4.0|        Pretty Color|The polish was qu...|[{large_image_url...|B00R8DXL44| B00R8DXL44|AGMJ3EMDVL6OWBJF7...|1598567408138|           0|             true|            1|  5|https://images-na...|[https://images-n...|
|   3.0|                 Meh|These were lightw...|[{large_image_url...|B088SZDGXG| B08BBQ29N5|AEYORY2AVPMCPDV57...|1

In [42]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
import requests
from PIL import Image
import numpy as np
import cv2
from io import BytesIO
import json

# Define schema for the result columns
schema = StructType([
    StructField("resolution", FloatType(), True),
    StructField("sharpness", FloatType(), True),
    StructField("brightness", FloatType(), True),
    StructField("contrast", FloatType(), True)
])

# Define UDF to process each row of the DataFrame
@pandas_udf(schema, PandasUDFType.SCALAR)
def analyze_images(images_col):
    resolutions = []
    sharpness_values = []
    brightness_values = []
    contrast_values = []

    for images in images_col:
        try:
            # Load images from URLs
            img_url = images[0]  # First image URL

            # Download the image
            response = requests.get(img_url)
            img = Image.open(BytesIO(response.content))

            # Compute resolution
            width, height = img.size
            resolution = width * height

            # Convert to grayscale for analysis
            image_cv = np.array(img.convert('L'))

            # Sharpness (Laplacian Variance)
            sharpness = cv2.Laplacian(image_cv, cv2.CV_64F).var()

            # Brightness (mean pixel value)
            brightness = np.mean(image_cv)

            # Contrast (max - min pixel value)
            contrast = image_cv.max() - image_cv.min()

            # Append results to lists
            resolutions.append(resolution)
            sharpness_values.append(sharpness)
            brightness_values.append(brightness)
            contrast_values.append(contrast)

        except:
            # Handle errors
            resolution = None
            sharpness = None
            brightness = None
            contrast = None

    # Return as pandas Series
    return pd.DataFrame({
        "resolution": resolutions,
        "sharpness": sharpness_values,
        "brightness": brightness_values,
        "contrast": contrast_values
    })

# Apply the UDF on the Spark DataFrame
pyspark_df_reviews = pyspark_df_reviews.withColumn("analysis", analyze_images(col("all_large_image_urls")))

# Replace None values with 0 (or a default value) in relevant columns
pyspark_df_reviews = pyspark_df_reviews.withColumn("analysis.resolution", when(col("analysis.resolution").isNull(), 0).otherwise(col("analysis.resolution")))\
                                        .withColumn("analysis.sharpness", when(col("analysis.sharpness").isNull(), 0).otherwise(col("analysis.sharpness")))\
                                        .withColumn("analysis.brightness", when(col("analysis.brightness").isNull(), 0).otherwise(col("analysis.brightness")))\
                                        .withColumn("analysis.contrast", when(col("analysis.contrast").isNull(), 0).otherwise(col("analysis.contrast")))

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [43]:
from pyspark.sql.functions import col, udf, when
from pyspark.sql.types import FloatType, IntegerType, StringType

# Define weights for each metric
WEIGHTS = {
    'resolution': 0.4,  # 40% weight
    'sharpness': 0.3,   # 30% weight
    'brightness': 0.2, # 15% weight
    'contrast': 0.1    # 15% weight
}

# Granular classification functions for resolution, sharpness, brightness, and contrast
def classify_by_resolution(resolution):
      if resolution < 1_000_000:
          return 2
      elif 1_000_000 <= resolution <= 2_000_000:
          return 6
      else:
          return 10

def classify_by_sharpness(sharpness):
    if sharpness is None:
        return 0
    if sharpness < 500:
        return 2
    elif 500 <= sharpness <= 1000:
        return 6
    else:
        return 10

def classify_by_brightness(brightness):
    if brightness is None:
        return 0
    if brightness < 50 or brightness > 200:
        return 2
    elif 50 <= brightness <= 100 or 150 <= brightness <= 200:
        return 6
    else:
        return 10

def classify_by_contrast(contrast):
    if contrast is None:
        return 0
    if contrast < 50:
        return 2
    elif 50 <= contrast <= 150:
        return 6
    else:
        return 10

# Register the classification functions as UDFs
classify_resolution_udf = udf(classify_by_resolution, IntegerType())
classify_sharpness_udf = udf(classify_by_sharpness, IntegerType())
classify_brightness_udf = udf(classify_by_brightness, IntegerType())
classify_contrast_udf = udf(classify_by_contrast, IntegerType())

# Add classification columns to the DataFrame
df_with_classification = pyspark_df_reviews \
    .withColumn("resolution_score", classify_resolution_udf(col("analysis.resolution")) * WEIGHTS['resolution']) \
    .withColumn("sharpness_score", classify_sharpness_udf(col("analysis.sharpness")) * WEIGHTS['sharpness']) \
    .withColumn("brightness_score", classify_brightness_udf(col("analysis.brightness")) * WEIGHTS['brightness']) \
    .withColumn("contrast_score", classify_contrast_udf(col("analysis.contrast")) * WEIGHTS['contrast'])

# Calculate the total weighted score for each row
df_with_scores = df_with_classification.withColumn(
    "image_quality",
    f.round((col("resolution_score") + col("sharpness_score") + col("brightness_score") + col("contrast_score")) / (sum(WEIGHTS.values()) * 10), 2)
)

# Select the relevant columns and show the result
df_with_scores.select(
    "title", "rating", "helpful_vote", "analysis.resolution", "analysis.sharpness",
    "analysis.brightness", "analysis.contrast", "image_quality"
).show(10, truncate=False)


+-----------------------------------+------+------------+----------+---------+----------+--------+-------------+
|title                              |rating|helpful_vote|resolution|sharpness|brightness|contrast|image_quality|
+-----------------------------------+------+------------+----------+---------+----------+--------+-------------+
|Pretty Color                       |4.0   |0           |1920000.0 |22.521935|102.756836|250.0   |0.6          |
|Meh                                |3.0   |0           |1218816.0 |130.40286|115.54641 |254.0   |0.6          |
|Nice colors and dries fast         |5.0   |0           |1371200.0 |49.51126 |107.86398 |255.0   |0.6          |
|Absolutely beautiful               |5.0   |0           |1892800.0 |787.8953 |182.21758 |255.0   |0.64         |
|Doesn't Sting or Burn My Eyes!     |5.0   |8           |512740.0  |55.262444|140.95502 |226.0   |0.44         |
|Best Peeling Kit I've Tried So Far |4.0   |0           |512120.0  |72.537834|72.96845  |190.0  

In [44]:
# df_with_scores.select(f.corr('rating', 'image_quality')).show()

In [45]:
df_final = df_with_scores.select(
    "title", "rating", "helpful_vote", 'asin', 'parent_asin', 'timestamp', 'helpful_vote', 'verified_purchase', 'number_images', "analysis.resolution", "analysis.sharpness",
    "analysis.brightness", "analysis.contrast", "image_quality"
)

In [46]:
# import pyodbc
# conn = pyodbc.connect('Driver={ODBC Driver 18 for SQL Server};Server=mit805assignment.database.windows.net;Database=dbo;UID=reogovender@deloitte.co.za;PWD=ChemEng20@23!7;Authentication=ActiveDirectoryPassword')
# cursor = conn.cursor()

In [47]:
# def generate_create_table_query(df, table_name):
#     schema = df.schema
#     column_definitions = []

#     for field in schema.fields:
#         # Map PySpark types to SQL types
#         if isinstance(field.dataType, IntegerType):
#             sql_type = "INT"
#         elif isinstance(field.dataType, StringType):
#             sql_type = "VARCHAR(255)"
#         else:
#             sql_type = "VARCHAR(255)"  # Default type

#         column_definitions.append(f"{field.name} {sql_type}")

#     columns_sql = ", ".join(column_definitions)
#     create_table_query = f"CREATE TABLE {table_name} ({columns_sql});"
#     return create_table_query

# # Generate the CREATE TABLE SQL query
# table_name = "new_table_1"
# create_query = generate_create_table_query(x, table_name)
# print(create_query)


# cursor.execute(create_query)
# conn.commit()

In [48]:
# def insert_data_from_df(df, table_name):
#     """Insert data into SQL Server using itertuples()."""
#     pdf = df.limit(10).toPandas()  # Convert to Pandas DataFrame (ensure memory fits)

#     # Build the INSERT query dynamically
#     columns = ", ".join(pdf.columns)
#     placeholders = ", ".join(["?"] * len(pdf.columns))
#     insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

#     # Insert data row by row using itertuples() for efficiency
#     for row in pdf.itertuples(index=False):
#         cursor.execute(insert_query, *row)

#     conn.commit()
#     print(f"Data inserted into {table_name} successfully.")

# insert_data_from_df(x, table_name)

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
df_final.count().cache()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 56368)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-50-8ea44c46d933>", line 1, in <cell line: 1>
    df_final.count().cache()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py", line 1240, in count
    return int(self._jdf.count())
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
  File "/usr/local/lib/python3.10/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprinta

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
df_final.write.parquet("/content/drive/MyDrive/test/testing.parquet", mode='overwrite')